# Training Procedure

In [ ]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
!rm -r ./mnist

!wget https://www.dropbox.com/s/5yre1ofqco5titj/mnist.zip?dl=0 -O mnist.zip
!unzip -q mnist.zip

!ls

## Setting data loader

Creating data loader for reading images from the training and test sets.

In [ ]:
import os
import numpy as np
import torch

from torch.utils.data import DataLoader
from torch.utils import data
from skimage import io

# Constants.
num_classes = 10
root = './mnist/'

# Class that reads a sequence of image paths from a text file and creates a data.Dataset with them.
class CustomDataset(data.Dataset):

    def __init__(self, fold, normalization = 'default'):

        super(CustomDataset, self).__init__()
        
        # Initializing variables.
        self.fold = fold
        self.normalization = normalization

        # Creating list of paths.
        self.imgs = self.make_dataset()

        # Check for consistency in list.
        if len(self.imgs) == 0:

            raise (RuntimeError('Found 0 images, please check the data set'))

    def make_dataset(self):

        # Initiating item list.
        items = []

        # Joining input paths.
        img_path = os.path.join(root, self.fold)

        # Reading paths from text file.
        #data_list = [l.strip('\n') for l in open(os.path.join(root, self.dataset, self.task + '_' + mode_str + '_f' + self.fold + '.txt')).readlines()]

        # Reading paths from directory.
        data_list = [f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))]
        
        # Creating list containing image and ground truth paths.
        for it in data_list:
            item = os.path.join(img_path, it)
            items.append(item)

        # Returning list.
        return items

    def __getitem__(self, index):

        # Reading items from list.
        img_path = self.imgs[index]
        
        # Reading images.
        img = io.imread(img_path)
        
        # Reading label from image file.
        lab = int(img_path[-5])
        
        # Removing unwanted channels. For the case of RGB images.
        if len(img.shape) > 2:
            img = img[:, :, 0] # Leaving only red (index = 0) channel from RGB.
        
        # Casting images to the appropriate dtypes.
        img = img.astype(np.float32)
        
        # Normalization.
        mn = img.min()
        mx = img.max()
        img = ((img - mn) / (mx - mn))
        
        # Adding channel dimension.
        img = np.expand_dims(img, axis=0)
        
        # Turning to tensors.
        img = torch.from_numpy(img)
        
        # Returning image and label to iterator.
        return img, lab

    def __len__(self):

        return len(self.imgs)

# Setting data loader.
batch_size = 100
num_workers = 2 # Number of Threads on each data_loader.

train_set = CustomDataset('train')
train_loader = DataLoader(train_set, batch_size, num_workers=num_workers, shuffle=True)

test_set = CustomDataset('test')
test_loader = DataLoader(test_set, batch_size, num_workers=num_workers, shuffle=False)

print('Size of training set: ' + str(len(train_set)) + ' samples')
print('Size of test set: ' + str(len(test_set)) + ' samples')


## Setting architecture

Creating a common CNN architecture composed of Convolutional and Fully Connected Layers.

Links úteis:

Função view() que dá reshape no tensor para adequá-lo ao tamanho da entrada das camadas. https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view

Pacote torch.nn que contém as implementações das camadas. https://pytorch.org/docs/stable/nn.html

In [ ]:
import torch.nn as nn

# Customized Network.
class CustomNetwork(nn.Module):
    
    def __init__(self, in_channels, num_classes=10):

        super(CustomNetwork, self).__init__()
        
        # TO DO: Implementar a arquitetura de uma MLP.
        # Exemplo de camada: self.layer1 = nn.Linear(args)
        
        
        self.initialize_weights()
    
    # Function for randomly initializing weights.
    def initialize_weights(self):
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        
        # TO DO: Implementar o forward.
        # Deve retornar a saída da última camada da rede.
        # A variável 'x' de entrada tem dimensões (BATCH_SIZE, N_CHANNELS, WIDTH, HEIGHT) = (100, 1, 28, 28).
        # A entrada de uma camada fully connected deve ter dimensões (BATCH_SIZE, N_INPUTS) = (100, 1*28*28).
        # A saída deve ter dimensões (BATCH_SIZE, N_CLASSES) = (100, 10).
        # Exemplo de forward em uma camada: out1 = self.layer1(x)
        # Um bom exercício é verificar o tamanho de todas as saídas da rede neural. Para verificar o tamanho de um tensor 'a', usar a função size: print(a.size()).
        
# Instancing Network.
in_channels = 1 # The input images only contain 1 channel.
num_classes = 10 # MNIST has 10 classes.

# model = CustomNetwork(in_channels, num_classes) # CPU version.
model = CustomNetwork(in_channels, num_classes).cuda() # GPU casting.


## Setting optimizer

$Pytorch$ has several options for optimizers, since the traditional SGD to more complex per-parameter adaptive ones (i.e. Adam, Adagrad, RSMProp...). All of them are located in the package torch.optim.

In [ ]:
import torch.optim as optim

lr = 0.0001 # Learning rate.
l2_normalization = 0.001 # L2 Normalization vai weight decay.

opt = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999), weight_decay=l2_normalization)

## Setting loss criterion

$CrossEntropyLoss$, as this is a classification task.

In [ ]:
# Setting a classification loss.
# criterion = nn.CrossEntropyLoss() # CPU version.
criterion = nn.CrossEntropyLoss().cuda() # GPU casting.

## Training/Testing

Iterating over epochs and batches.

In [ ]:
from matplotlib import pyplot as plt
from torch.autograd import Variable

%matplotlib inline

epochs = 20 # Run network for 20 epochs.

training_metrics = list() # List for accuracies in training procedure.
test_metrics = list() # List for accuracies in test procedure.

# Iterating over epochs.
for ep in range(epochs):
    
    print('##############################################')
    print('Starting epoch ' + str(ep + 1) + '/' + str(epochs) + '...')
    
    #####################################################################
    # Training Procedure. ###############################################
    #####################################################################
    
    print('    Training...')
    
    # Setting model to training mode.
    model.train()
    
    total_correct = 0
    
    # Iterating over training batches.
    for it, data in enumerate(train_loader):

        # Obtaining images and labels for batch.
        inps, labs = data
        
        # GPU casting. In CPU version, remove the following two lines.
        inps = Variable(inps.cuda(), requires_grad=True)
        labs = Variable(labs.cuda(), requires_grad=False) # requires_grad = False -> it isn't necessary to compute gradients from targets, only from losses.
        
        # Zeroing optimizer.
        opt.zero_grad()
        
        # Forwarding inputs through DNN.
        output = model(inps)
        
        # Computing loss according to network prediction for batch and targets.
        # The Cross Entropy loss receives an output of size (BATCH_SIZE, N_CLASSES) and a label vector of size (BATCH_SIZE).
        loss = criterion(output, labs)
        
        # Backpropagating loss.
        loss.backward() # All backward pass is computed from this line automatically by package torch.autograd.
        
        # Taking optimization step.
        opt.step()
        
        # Computing prediction to batch
        pred = output.max(1, keepdim=True)[1]
        
        # Computing accuracy for batch.
        correct = pred.eq(labs.view_as(pred)).sum().item()
        accuracy = (100.0 * float(correct) / inps.size(0))
        
        training_metrics.append(accuracy)
        
        # Updating total counter.
        total_correct += correct
        
    # Computing accuracy for whole epoch.
    accuracy = 100.0 * float(total_correct) / len(train_set)
    
    print('        Accuracy for training epoch [' + str(ep + 1) + ' / ' + str(epochs) + ']: ' + str(accuracy))
    
    #####################################################################
    # Testing Procedure.  ###############################################
    #####################################################################
    
    print('    Testing...')
        
    # Setting model to evaluation mode.
    model.eval()
    
    # GPU casting. In CPU version, remove the following two lines.
    #inps = Variable(inps.cuda(), requires_grad=True)
    #labs = Variable(labs.cuda(), requires_grad=True)
    
    total_correct = 0

    # Iterating over test batches.
    for it, data in enumerate(test_loader):

        # Obtaining images and labels for batch.
        inps, labs = data
        
        # GPU casting. In CPU version, remove the following line.
        inps = Variable(inps.cuda(), requires_grad=True)
        labs = Variable(labs.cuda(), requires_grad=False)
        
        # Forwarding through DNN.
        output = model(inps)
        
        # Computing prediction to batch
        pred = output.max(1, keepdim=True)[1]
        
        # Computing accuracy for batch.
        correct = pred.eq(labs.view_as(pred)).sum().item()
        accuracy = (100.0 * float(correct) / inps.size(0))
        
        # Appending accuracy in list of accuracies for all batches.
        test_metrics.append(accuracy)
        
        # Updating total counter.
        total_correct += correct
        
    # Computing accuracy for whole epoch.
    accuracy = 100.0 * float(total_correct) / len(test_set)
    
    print('        Accuracy for test epoch [' + str(ep + 1) + ' / ' + str(epochs) + ']: ' + str(accuracy))
    
# Transforming list into ndarray for plotting.
training_array = np.asarray(training_metrics, dtype=np.float32)
test_array = np.asarray(test_metrics, dtype=np.float32)

# Plotting accuracy.
fig, ax = plt.subplots(1, 2, figsize = (16, 8), sharex = False, sharey = True)

training_plt = ax[0].plot(training_array)
ax[0].set_xlabel('Training')
ax[0].set_ylim([0, 100])

test_plt = ax[1].plot(test_array)
ax[1].set_xlabel('Test')
ax[1].set_ylim([0, 100])

plt.show()